# HELIX-SCOPE

### Developing the data processing

In this notebook we will test how best to process national summary statics from the Helix consortium data. Summary statistics (mean, max, min and standard deviation) will be calculated for every shape in an arbitrary shapefile for every netcdf file on path.

Data should be downloaded from the SFTP site (bi.nsc.liu.se), which requires a username and password login. The data should be placed in the `/data` folder within this repo.

** THIS NOTEBOOK IS FOR DEVELOPMENT PURPOSES ONLY**

In [1]:
from netCDF4 import Dataset
import os
import cartoframes
import re
import fiona
import rasterio
from rasterio.mask import mask
from rasterio.plot import show
from rasterstats import zonal_stats
import geopandas as gpd
import pandas as pd
import numpy as np
from matplotlib.pyplot import cm
import matplotlib.pyplot as plt
import datetime
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

/opt/conda/lib/python3.5/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
/opt/conda/lib/python3.5/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
def identify_netcdf_and_csv_files(path='data'):
    """Crawl through a specified folder and return a dict of the netcdf d['nc']
    and csv d['csv'] files contained within.
    Returns something like {'nc':'data/CNRS_data/cSoil/orchidee-giss-ecearth.SWL_15.eco.cSoil.nc'}
    """
    netcdf_files = []
    csv_files = []
    for root, dirs, files in os.walk(path):
        if isinstance([], type(files)):
            for f in files:
                if f.split('.')[-1] in ['nc']:
                    netcdf_files.append(''.join([root,'/',f]))
                elif  f.split('.')[-1] in ['csv']:
                    csv_files.append(''.join([root,'/',f]))
    return {'nc':netcdf_files,'csv':csv_files}


def generate_metadata(filepath):
    """Pass a path and file as a sigle string. Expected in the form of:
        data/CNRS_data/cSoil/orchidee-giss-ecearth.SWL_15.eco.cSoil.nc
    """
    file_metadata = get_nc_attributes(filepath)
    filename_properties = extract_medata_from_filename(filepath)
    return {**file_metadata, **filename_properties}


def extract_medata_from_filename(filepath):
    """extract additonal data from filename using REGEX"""
    warning = "Filepath should resemble: data/CNRS_data/cSoil/orchidee-giss-ecearth.SWL_15.eco.cSoil.nc"
    assert len(file.split('/')) == 4, warning
    fname = filepath.split("/")[3]
    variable = filepath.split("/")[2]
    model_taxonomy = re.search('(^.*?)\.',fname, re.IGNORECASE).group(1)
    model_short_name = re.search('(^.*?)-',model_taxonomy, re.IGNORECASE).group(1)
    return {"model_short_name":model_short_name, "variable":variable, "model_taxonomy":model_taxonomy}


def get_nc_attributes(filepath):
    """ Most info is stored in the files’ global attribute description,
    we will access it using netCDF4.ncattrs function.
    Example:
         ncAttributes('data/CNRS_data/cSoil/orchidee-giss-ecearth.SWL_15.eco.cSoil.nc')
    """
    nc_file = Dataset(filepath, 'r')
    d = {}
    nc_attrs = nc_file.ncattrs()    
    for nc_attr in nc_attrs:
        d.update({nc_attr: nc_file.getncattr(nc_attr)})
    could_be_true = ['true', 'True', 'TRUE']
    d['is_multi_model_summary'] = d['is_multi_model_summary'] in could_be_true
    d['is_seasonal'] = d['is_seasonal'] in could_be_true
    del d['contact']
    return d


def get_shape_attributes(i):
    """Get attributes of shapes for gadm28_admin1 data
    index (i) should be passed
    """
    d = {}
    for table_attribute in ['iso','name_0','id_1','name_1','engtype_1']:
        try:
            d[table_attribute] = shps[table_attribute][i]
        except:
            d[table_attribute] = None
    return d

## Single core process

Single core version:

Place the data folders from Helixscope into the data folder of this repo.

```
data
├── CNRS_data
│   ├── README.txt
│   ├── cSoil
│   │   ├── orchidee-giss-ecearth.SWL_15.eco.cSoil.nc
│   │   ├── orchidee-giss-ecearth.SWL_2.eco.cSoil.nc
│   │   ├── orchidee-giss-ecearth.SWL_4.eco.cSoil.nc
│   │   ├── orchidee-ipsl-ecearth.SWL_15.eco.cSoil.nc
│   │   ├── orchidee-ipsl-ecearth.SWL_2.eco.cSoil.nc
│   │   ├── orchidee-ipsl-ecearth.SWL_4.eco.cSoil.nc
│   │   ├── orchidee-ipsl-hadgem.SWL_15.eco.cSoil.nc
│   │   ├── orchidee-ipsl-hadgem.SWL_2.eco.cSoil.nc
│   │   └── orchidee-ipsl-hadgem.SWL_4.eco.cSoil.nc
│   ├── cVeg
│   │   ├── orchidee-giss-ecearth.SWL_15.eco.cVeg.nc
│   │   ├── orchidee-giss-ecearth.SWL_2.eco.cVeg.nc
│   │   ├── orchidee-giss-ecearth.SWL_4.eco.cVeg.nc
│   │   ├── orchidee-ipsl-ecearth.SWL_15.eco.cVeg.nc
│   │   ├── orchidee-ipsl-ecearth.SWL_2.eco.cVeg.nc
```

Also include the shapefile in the data folder:

```
./data/minified_gadm28_countries/gadm28_countries.shp
```

In [4]:
%%time
#shps = gpd.read_file('./data/minified_gadm28_countries/gadm28_countries.shp')
shps = gpd.read_file('./data/gadm28_adm1/gadm28_adm1.shp')
shps = shps.to_crs(epsg='4326')
files = identify_netcdf_and_csv_files()

keys = ['name_0','iso','id_1','name_1','engtype_1','variable','SWL_info',
        'count', 'max','min','mean','std','impact_tag','institution',
        'model_long_name','model_short_name','model_taxonomy',
        'is_multi_model_summary','is_seasonal']

CPU times: user 1min 12s, sys: 360 ms, total: 1min 12s
Wall time: 1min 13s


In [74]:
def process_file(file, status=False, overwrite=False):
    """Given a single file, generate a csv table with the same folder/file name
    in ./data/processed/ with all required csv info.
    Expect file to be a string e.g.: "data/CNRS_data/cSoil/orchidee-giss-ecearth.SWL_15.eco.cSoil.nc"
    """
    output_filename = "".join(['./processed/',file[5:-3],'.csv'])
    if os.path.isfile(output_filename) and not overwrite:
        print("{0} output exists.".format(output_filename))
        print("Specifcy overwrite=True on process_file() function call if you want to replace it.")
        return
    else:
        if status: print("Processing '{}'".format(file))
        tmp_metadata = generate_metadata(file)
        with rasterio.open(file) as nc_file:
            rast=nc_file.read()
            properties = nc_file.profile
        tmp = rast[0,:,:]                      
        mask = tmp == properties.get('nodata') 
        tmp[mask] = np.nan
        stats_per_file = []
        for i in shps.index:
            shp = shps.iloc[i].geometry
            zstats = zonal_stats(shp, tmp, band=1, stats=['mean', 'max','min','std','count'],
                                 all_touched=True, raster_out=False,
                                 affine=properties['transform'],
                                 no_data=np.nan)
            if zstats[0].get('count', 0) > 0:
                shp_atts = get_shape_attributes(i)
                tmp_d = {**zstats[0], **shp_atts, **tmp_metadata}
                stats_per_file.append([tmp_d.get(key, None) for key in keys])
        df = pd.DataFrame(stats_per_file, columns=keys)
        path_check = "/".join(output_filename.split("/")[0:-1])
        if not os.path.exists(path_check):
            os.makedirs(path_check)
        df.to_csv(output_filename, index=False)
        return
    
    
def combine_processed_results(path='./processed'):
    """Combine all the csv files in the path (e.g. all processed files)
    into a single master table
    """
    output_files = identify_netcdf_and_csv_files(path)
    frames = [pd.read_csv(csv_file) for csv_file in output_files['csv']]
    master_table = pd.concat(frames)
    master_table.to_csv("./master_admin1.csv", index=False)
    return

In [78]:
for file in files.get('nc')[0:3]:
    print(file)
    process_file(file)

data/CNRS_data/cSoil/orchidee-giss-ecearth.SWL_15.eco.cSoil.nc


data/CNRS_data/cSoil/orchidee-giss-ecearth.SWL_2.eco.cSoil.nc
data/CNRS_data/cSoil/orchidee-giss-ecearth.SWL_4.eco.cSoil.nc
./processed/CNRS_data/cSoil/orchidee-giss-ecearth.SWL_4.eco.cSoil.csv output exists.
Specifcy overwrite=True on process_file() function call if you want to replace it.


In [80]:
output_files['csv']

['./processed/CNRS_data/cSoil/orchidee-giss-ecearth.SWL_15.eco.cSoil.csv',
 './processed/CNRS_data/cSoil/orchidee-giss-ecearth.SWL_2.eco.cSoil.csv',
 './processed/CNRS_data/cSoil/orchidee-giss-ecearth.SWL_4.eco.cSoil.csv']

In [94]:
def combine_processed_results(path='./processed'):
    """Combine all the csv files in the path (e.g. all processed files)
    into a single master table
    """
    output_files = identify_netcdf_and_csv_files(path)
    frames = [pd.read_csv(csv_file) for csv_file in output_files['csv']]
    master_table = pd.concat(frames)
    master_table.to_csv("./master_admin1.csv", index=False)
    return

,name_0,iso,id_1,name_1,engtype_1,variable,SWL_info,count,max,min,mean,std,impact_tag,institution,model_long_name,model_short_name,model_taxonomy,is_multi_model_summary,is_seasonal
0,Mexico,MEX,9,Ciudad de México,Federal District,cSoil,1.5,3,9.999629,5.903975,8.440266,1.809122,eco,LSCE,ORCHIDEE,orchidee,orchidee-giss-ecearth,False,False
1,Haiti,HTI,3,L'Artibonite,Department,cSoil,1.5,6,4.599074,1.839996,3.358963,0.888974,eco,LSCE,ORCHIDEE,orchidee,orchidee-giss-ecearth,False,False
2,Honduras,HND,1,Atlántida,Department,cSoil,1.5,5,7.771445,5.581614,6.300231,0.789679,eco,LSCE,ORCHIDEE,orchidee,orchidee-giss-ecearth,False,False
3,Honduras,HND,3,Colón,Department,cSoil,1.5,9,7.171936,5.421208,6.181699,0.568947,eco,LSCE,ORCHIDEE,orchidee,orchidee-giss-ecearth,False,False
4,Guyana,GUY,9,Upper Demerara-Berbice,Region,cSoil,1.5,11,6.191456,4.655702,5.380422,0.418534,eco,LSCE,ORCHIDEE,orchidee,orchidee-giss-ecearth,False,False


In [5]:
%%time

for file in files.get('nc')[0:1]:
    print("Processing '{}'".format(file))
    tmp_metadata = generate_metadata(file)
    with rasterio.open(files['nc'][0]) as nc_file:
        rast=nc_file.read()
        properties = nc_file.profile
    tmp = rast[0,:,:]                      # The first dim should be stripped
    mask = tmp == properties.get('nodata') # Now we need to make a mask for missing data
    tmp[mask] = np.nan                     # and replace it with a NAN value
    stats_per_file = []
    for i in shps.index:
        shp = shps.iloc[i].geometry
        zstats = zonal_stats(shp, tmp, band=1, stats=['mean', 'max','min','std','count'],
                             all_touched=True, raster_out=False,
                             affine=properties['transform'],
                             no_data=np.nan)
        if zstats[0].get('count', 0) > 0: # If shape generated stats, then add it
            #shp_atts = {'iso2' : shps.iso2[i],
            #            'country' : shps.name_engli[i]}
            shp_atts = get_shape_attributes(i)
            tmp_d = {**zstats[0], **shp_atts, **tmp_metadata}
            stats_per_file.append([tmp_d.get(key, None) for key in keys])

Processing 'data/CNRS_data/cSoil/orchidee-giss-ecearth.SWL_15.eco.cSoil.nc'
CPU times: user 1min 47s, sys: 1.18 s, total: 1min 48s
Wall time: 1min 54s


In [27]:
df = pd.DataFrame(stats_per_file, columns=keys)
df.head()

,name_1,iso,id_1,name_1,engtype_1,variable,SWL_info,count,max,min,mean,std,impact_tag,institution,model_long_name,model_short_name,model_taxonomy,is_multi_model_summary,is_seasonal
0,Ciudad de México,MEX,9,Ciudad de México,Federal District,cSoil,1.5,3,9.999629,5.903975,8.440266,1.809122,eco,LSCE,ORCHIDEE,orchidee,orchidee-giss-ecearth,False,False
1,L'Artibonite,HTI,3,L'Artibonite,Department,cSoil,1.5,6,4.599074,1.839996,3.358963,0.888974,eco,LSCE,ORCHIDEE,orchidee,orchidee-giss-ecearth,False,False
2,Atlántida,HND,1,Atlántida,Department,cSoil,1.5,5,7.771445,5.581614,6.300231,0.789679,eco,LSCE,ORCHIDEE,orchidee,orchidee-giss-ecearth,False,False
3,Colón,HND,3,Colón,Department,cSoil,1.5,9,7.171936,5.421208,6.181699,0.568947,eco,LSCE,ORCHIDEE,orchidee,orchidee-giss-ecearth,False,False
4,Upper Demerara-Berbice,GUY,9,Upper Demerara-Berbice,Region,cSoil,1.5,11,6.191456,4.655702,5.380422,0.418534,eco,LSCE,ORCHIDEE,orchidee,orchidee-giss-ecearth,False,False


In [29]:
df.to_csv('./processed/raw_output.csv')

Next steps:

* Need to ensure this can handel any admin1 shapefiles
    - Done, but big slowdown (2 min per layer)
* Need to paraellise this so it will run in a convienient time
* Need to check that the regex changes Alex applied post-table creation are included

In [3]:
import cartoframes

In [ ]:
shps.keys()

In [ ]:
with open(".env") as f:
    key = f.read()
api_key = key.split('API_KEY=')[1].split()[0]

'helixscope'

In [25]:
CF = cartoframes.CartoContext(
    creds=cartoframes.Credentials(username='pycones03', key=api_key)
)

In [ ]:
CF.write(boundaries, 'uk_boundaries', overwrite=True)

In [ ]:
CF.map(layers=[
    cartoframes.BaseMap('light'),
    cartoframes.Layer('uk_boundaries'),
], interactive=True)

CF.map(layers=[
    cartoframes.BaseMap('light'),
    cartoframes.Layer('trump',
        color={'column': 'trump_haters',
               'scheme': cartoframes.styling.sunset(5)}),
], interactive=True)


uk_pop = [{'numer_id': 'uk.ons.LC2102EW0001', 'normalization': 'prenormalized'}]
augmented = CF.data_augment('trump', uk_pop)
augmented.head()